# Gathering links of the paintings

### Step1 : extract links from each 'query' (page of the website with fine art paintings)

In [4]:


import csv
import urllib2 as ul   #url.request lib for handling the url
from bs4 import BeautifulSoup #bs for parsing the page

def get_painting_urls(soup, links):
                                        #print soup.h4.a.get('href') EQUAL
    for tag in soup.find_all('h4'):     # class name that contains all the links to the paintings
        try:
            for item in tag:            # for each element with that "class name" we look for the href (link)
                try: 
                    if item.get('href') != None:
                        links.append([item.get('href')])   #list of links
                except:
                    pass
        except:
            pass

def write_painting_urls(wcsv, onelist):
        for i in onelist:
            wcsv.writerow(i)        
        
        
url_basic='http://www.saatchiart.com/paintings/fine-art?page='

links=list()
links_notduplicated=list()

### Create a file with the links received (subest of the first 10 pages and paintings that appear in the 'query' )

with open('paintinglinks.csv', 'w') as csvfile:
        linkswritten = csv.writer(csvfile, delimiter=',')
        
        for i in range(10):                         
            url = url_basic + str(i+1)              # url that has all the data (query with 2375 pages on the website)
            url_response=ul.urlopen(url,timeout=5)  # open that url
            soup = BeautifulSoup(url_response)      # parse information
            get_painting_urls(soup, links)          # FUNCTION ADDING LINKS TO A LIST
            for each in links:
                if each not in links_notduplicated:
                    links_notduplicated.append(each)
            links = []  
            write_painting_urls(linkswritten, links_notduplicated) # FUNCTION PRINTING THE LIST INTO A FILE
            print "ready links in page ", i , " -> ", len(links_notduplicated) 
            links_notduplicated = []



ready links in page  0  ->  24
ready links in page  1  ->  24
ready links in page  2  ->  24
ready links in page  3  ->  24
ready links in page  4  ->  24
ready links in page  5  ->  24
ready links in page  6  ->  24
ready links in page  7  ->  24
ready links in page  8  ->  24
ready links in page  9  ->  24


# Step2: extract features of each paiting (link)

##### Define functions to be used while scrapping

In [6]:
def get_feature(soup, str_property, features_list):
    empty = 1
    for tag in soup.find_all('meta'):    # each time we find the meta class we access to each element to add it on a list
        try:
            if tag['property']==str_property:   # we take the "property" corresponding to the feature that we are looking for
                features_list.append(tag['content'])  # we ask for the feature of that property which is stored in a tag called content
                empty = empty * 0
        except:
            pass
    if empty == 1:
        features_list.append('NA')   # NA for missing information

def get_size_features(soup):
    stuff=list()
    for tag in soup.find_all('p'):       # the size is stored in another class 
        try:
            if tag['class']==['category-size']:
                for item in tag:
                    try: 
                        if str(item).find('/span')>0:      # we extract each element height and width 
                            stuff.append(item.get_text())  # we add those value on the list as more features
                    except:
                        pass
        except:
            pass 
    return stuff

def construct_feature_list(soup):
    features = list()
    get_feature(soup, 'bt:artist',features)   # calling method to extract feature (such as artist) of that painting
    string = ''
    for letter in features[0]:
        try:
            x=ord(letter)
            if x<65 or x>90:
                if x<97 or x>122:
                    if x!=32: x=42
                    else: pass
                else: pass
            else:pass
            string+=chr(x)
        except:
            x=42
            string+=chr(x)
    features[0]=string
    get_feature(soup, 'bt:artistCountry',features)
    get_feature(soup, 'bt:keywords',features)
    get_feature(soup, 'bt:subject',features)
    get_feature(soup, 'bt:styles',features)
    get_feature(soup, 'bt:mediums',features)
    get_feature(soup, 'bt:materials',features)
    get_feature(soup, 'bt:displayPrice',features)
    get_feature(soup, 'bt:category',features)
    stuff = get_size_features(soup) #after adding each feature, we ask for the size of the painting
    features = features + stuff  # features and size in a unique line
    return features


##### Use the csv file to go to each painting-link and get metadata

Example: [https://www.saatchiart.com/art/Painting-Good-Morning-Francis/669812/3078190/view](Painting link)

In [7]:

import csv
import urllib2 as ul #url.request lib for handling the url
from bs4 import BeautifulSoup #bs to parse

j = 0
unwritten = []
line_number_unopened = []
j = 1
number_unopened = 0

url_basic = 'http://www.saatchiart.com'

## Create a new file with all the informtion available
with open('painting_features_table.csv', 'w') as csvfile2:
    
    featureswritten = csv.writer(csvfile2, delimiter=',')
    headings = ['link', 'artist', 'artist_country', 'keywords', 'subject', 'styles', 'mediums', 'materials', 'list_price', 'category', 'height', 'width', 'depth']
    featureswritten.writerow(headings)      # heading for the file that will contain columns with data from the paintings
    
    with open('paintinglinks.csv') as csvfile:
        linkswritten = csv.reader(csvfile, delimiter=',')        
        for line in linkswritten:
            if j>0 and j<=1000 and number_unopened<=100:  # Is better to run into batches this because websites might have rules to stop the server
                url=url_basic+line[0]                 
                try:
                    url_response=ul.urlopen(url,timeout=5)
                    soup = BeautifulSoup(url_response)
                    features = construct_feature_list(soup)    # from each painting go to this method to get the features (columns)
                    features = line + features                 # for each line (paiting that is identified by a link) we include the features
                    try:        
                        featureswritten.writerow(features)   #writing everything in the file of features
                    except:
                        print("Couldn't write row! or maybe I've finished, line -> ", line)
                        unwritten.append(j)
                except:
                    number_unopened += 1
                    line_number_unopened.append(j)
                    print("Couldn't open url!")
                j+=1
            elif j<=0:
                j+=1
            else:
                print(j)
                break

print "ready"


("Couldn't write row! or maybe I've finished, line -> ", ['/art/Painting-Hammer-pictures-49/891427/3453985/view'])
ready


##### This is how the data looks like

In [10]:
import pandas as pd
pd.read_csv('painting_features_table.csv').head(5)

,link,artist,artist_country,keywords,subject,styles,mediums,materials,list_price,category,height,width,depth
0,/art/Painting-Uncanny-I/155384/2488851/view,Yiota Kotika,Greece,"sad, tears, no, eyes, face, lips",Portrait,"Fine Art, Photorealism, Realism","Acrylic, pencils",Canvas,$445.00,Painting,11.8 H,9.4 W,1.6
1,/art/Painting-Bipolar/311546/2121764/view,Michael Hayter,United Kingdom,"surreal, mental health, colour, fantasy, figur...",Body,"Figurative, Expressionism, Fine Art","Oil, Ink, Acrylic, Saliva, Urine","Canvas, Wood",$3350.00,Painting,71.7 H,70.9 W,2.0
2,/art/Painting-Ultramarine-Traces/486818/310758...,Aleksandra Toborowicz,Poland,"bird, blue, traces, ultramarine, white, abstra...",Abstract,"Abstract, Abstract Expressionism, Expressionis...","Acrylic, Watercolor",Paper,$5410.00,Painting,27.6 H,39.4 W,0.0
3,/art/Painting-Shaolin/993012/3671205/view,Aibek Begalin,United States,"shaolin, realism, fine art, oil painting, oil ...",Landscape,"Figurative, Fine Art, Photorealism, Realism, I...",Oil,Canvas,$1920.00,Painting,30 H,40 W,0.7
4,/art/Painting-Symbiosis-II/350258/3512342/view,Joanna Ml*cka,Poland,"red, square, composition, floral, forest, gree...",Floral,"Abstract, Fine Art, Realism, Surrealism, Figur...",Oil,Canvas,$760.00,Painting,15.7 H,15.7 W,1.0
